#Sarcasm Detection (sarcastic or not)




Download the dataset of sarcastic and non sarcastic tweets with emojis and splitting it into train and test sets



In [ ]:
! gdown 1X1jZgrRaVH-h0xX3k8cSzT0kumIVlmwx #sarc_emojis_1000.csv
! gdown 1--LLxkODAryG3TgUHrcggfrYoXnr9UFe #not_sarc_emojis_1000.csv

Downloading...
From: https://drive.google.com/uc?id=1X1jZgrRaVH-h0xX3k8cSzT0kumIVlmwx
To: /content/sarc_emojis_1000.csv
100% 1.60M/1.60M [00:00<00:00, 92.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--LLxkODAryG3TgUHrcggfrYoXnr9UFe
To: /content/not_sarc_emojis_1000.csv
100% 1.68M/1.68M [00:00<00:00, 43.7MB/s]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

sarcastic_df = pd.read_csv('sarc_emojis_1000.csv')
non_sarcastic_df = pd.read_csv('not_sarc_emojis_1000.csv')

# Append emoticon to text in both datasets
sarcastic_df['text'] = sarcastic_df['text'] + ' ' + sarcastic_df['label']
non_sarcastic_df['text'] = non_sarcastic_df['text'] + ' ' + non_sarcastic_df['label']

# Add labels to the data
sarcastic_df['class'] = 1  # 1 for sarcastic
non_sarcastic_df['class'] = 0  # 0 for non sarcastic

# Prepare empty dataframes for training and testing sets
train_df = pd.DataFrame()
test_df = pd.DataFrame()

# Assuming 'label' is the column with the label information
labels = sarcastic_df['label'].unique()  # or a predefined list of labels if this is not accurate

for label in labels:
    # Filter sarcastic and non-sarcastic dataframes by label
    sarcastic_label_df = sarcastic_df[sarcastic_df['label'] == label]
    non_sarcastic_label_df = non_sarcastic_df[non_sarcastic_df['label'] == label]

    # Split sarcastic data for the current label into train and test
    sarcastic_train = sarcastic_label_df.sample(frac=0.8, random_state=42)
    sarcastic_test = sarcastic_label_df.drop(sarcastic_train.index)

    # Split non-sarcastic data for the current label into train and test
    non_sarcastic_train = non_sarcastic_label_df.sample(frac=0.8, random_state=42)
    non_sarcastic_test = non_sarcastic_label_df.drop(non_sarcastic_train.index)

    # Append splits to the corresponding training and testing dataframes
    train_df = pd.concat([train_df, sarcastic_train, non_sarcastic_train], ignore_index=True)
    test_df = pd.concat([test_df, sarcastic_test, non_sarcastic_test], ignore_index=True)

# Shuffle the final train and test dataframes to ensure a mix of labels
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract features and labels for training and testing
X_train, y_train = train_df['text'], train_df['class']
X_test, y_test = test_df['text'], test_df['class']

In [ ]:
def calc_and_print_eveluation_metrics(y_test, y_pred):
  print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
  print(f"Precision: {precision_score(y_test, y_pred)}")
  print(f"Recall: {recall_score(y_test, y_pred)}")
  print(f"F1 Score: {f1_score(y_test, y_pred)}")

##Unigrams

In [ ]:
# Create a CountVectorizer to convert text into a bag-of-words representation (unigrams)
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

###SVM unigrams

In [ ]:
# Build an SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_vectorized)

# Calculate and print eveluation metrics
print('SVM unigrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)

SVM unigrams results:

Accuracy: 0.7225
Precision: 0.7226113056528264
Recall: 0.72225
F1 Score: 0.7224306076519129


###Random Forest unigrams

In [ ]:
# Build a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_vectorized)

# Calculate and print eveluation metrics
print('Random Forest unigrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)

Random Forest unigrams results:

Accuracy: 0.71725
Precision: 0.737431693989071
Recall: 0.67475
F1 Score: 0.7046997389033942


###Logistic Regression unigrams

In [ ]:
from sklearn.linear_model import LogisticRegression

# Build a Logistic Regression model
log_reg_model = LogisticRegression(random_state=42, max_iter=100)
log_reg_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = log_reg_model.predict(X_test_vectorized)

# Calculate and print evaluation metrics
print('Logistic Regression unigrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)


Logistic Regression unigrams results:

Accuracy: 0.731125
Precision: 0.7366265676990018
Recall: 0.7195
F1 Score: 0.7279625648159859


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##Ngrams - 2 chars

In [ ]:
# Create a CountVectorizer to convert text into a bag-of-words representation (character n-grams)
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

###SVM ngrams

In [ ]:
# Build an SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_vectorized)

# Calculate and print eveluation metrics
print('SVM ngrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)

SVM ngrams results:

Accuracy: 0.69625
Precision: 0.7048538622129437
Recall: 0.67525
F1 Score: 0.6897344228804902


###Random Forest ngrams

In [ ]:
# Build a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_vectorized)

# Calculate and print eveluation metrics
print('Random Forest ngrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)

Random Forest ngrams results:

Accuracy: 0.71175
Precision: 0.7195438050803525
Recall: 0.694
F1 Score: 0.7065411046067702


###Logistic Regression ngrams

In [ ]:
from sklearn.linear_model import LogisticRegression

# Build a Logistic Regression model
log_reg_model = LogisticRegression(random_state=42, max_iter=1000)
log_reg_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = log_reg_model.predict(X_test_vectorized)

# Calculate and print evaluation metrics
print('Logistic Regression n-grams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)


Logistic Regression n-grams results:

Accuracy: 0.6935
Precision: 0.6976506639427987
Recall: 0.683
F1 Score: 0.6902475997978778


##Ngrams - 3 chars

In [ ]:
# Create a CountVectorizer to convert text into a bag-of-words representation (character n-grams)
vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

###SVM ngrams

In [ ]:
# Build an SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_vectorized)

# Calculate and print eveluation metrics
print('SVM ngrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)

SVM ngrams results:

Accuracy: 0.727875
Precision: 0.7289625722180356
Recall: 0.7255
F1 Score: 0.7272271645157248


###Random Forest ngrams

In [ ]:
# Build a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_vectorized)

# Calculate and print eveluation metrics
print('Random Forest ngrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)

Random Forest ngrams results:

Accuracy: 0.736125
Precision: 0.7549257759784076
Recall: 0.69925
F1 Score: 0.7260220635950683


###Logistic Regression ngrams

In [ ]:
from sklearn.linear_model import LogisticRegression

# Build a Logistic Regression model
log_reg_model = LogisticRegression(random_state=42, max_iter=1000)
log_reg_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = log_reg_model.predict(X_test_vectorized)

# Calculate and print evaluation metrics
print('Logistic Regression n-grams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)


Logistic Regression n-grams results:

Accuracy: 0.741625
Precision: 0.7413233458177279
Recall: 0.74225
F1 Score: 0.741786383510306


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Ngrams - 2 words

In [ ]:
# Create a CountVectorizer to convert text into a bag-of-words representation (character n-grams)
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

###SVM ngrams

In [ ]:
# Build an SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_vectorized)

# Calculate and print eveluation metrics
print('SVM ngrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)

SVM ngrams results:

Accuracy: 0.68625
Precision: 0.7143268124280783
Recall: 0.62075
F1 Score: 0.664258962011771


###Random Forest ngrams

In [ ]:
# Build a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_vectorized)

# Calculate and print eveluation metrics
print('Random Forest ngrams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)

Random Forest ngrams results:

Accuracy: 0.6865
Precision: 0.75
Recall: 0.5595
F1 Score: 0.6408934707903781


###Logistic Regression ngrams

In [ ]:
from sklearn.linear_model import LogisticRegression

# Build a Logistic Regression model
log_reg_model = LogisticRegression(random_state=42, max_iter=1000)
log_reg_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = log_reg_model.predict(X_test_vectorized)

# Calculate and print evaluation metrics
print('Logistic Regression n-grams results:\n')
calc_and_print_eveluation_metrics(y_test, y_pred)


Logistic Regression n-grams results:

Accuracy: 0.7005
Precision: 0.7392601431980907
Recall: 0.6195
F1 Score: 0.6741022850924918


##Bert

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


texts = df['text'].tolist()
labels = df['class'].tolist()

# Load pre-trained DistilBERT model and tokenizer
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Assuming binary classification

# Tokenize and encode the input texts
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Create TensorDataset for training
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], torch.tensor(labels))

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Define data loaders
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Define optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 4

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        optimizer.zero_grad()

        input_ids, attention_mask, label = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_loss}')

# Evaluation on the test set
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, label = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).tolist()
        all_preds.extend(preds)
        all_labels.extend(label.tolist())

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print('BERT results: \n')
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

NameError: name 'df' is not defined

##RoBERTA

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

texts = df['text'].tolist()
labels = df['class'].tolist()

# Load pre-trained RoBERTa model and tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Assuming binary classification

# Tokenize and encode the input texts
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Create TensorDataset for training
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], torch.tensor(labels))

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Define data loaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Define optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 3

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        optimizer.zero_grad()

        input_ids, attention_mask, label = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_loss}')

# Evaluation on the test set
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, label = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).tolist()
        all_preds.extend(preds)
        all_labels.extend(label.tolist())

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
